<a href="https://colab.research.google.com/github/Hamza-Ali0237/PyTorch-Transformer-From-Scratch/blob/main/Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Encoder-Decoder Tranformer From Scratch Using PyTorch

Implemeting The Encoder-Decoder Transformer Architecture From The 2017 Paper Published By Google ["*Attention Is All You Need* "](https://arxiv.org/abs/1706.03762)

In [1]:
# Importing Libraries
import math
import torch
from torch import nn
import torch.nn.functional as F

In [2]:
class InputEmbeddings(nn.Module):
  def __init__(self, vocab_size, d_model):
    super().__init__()

    self.d_model = d_model
    self.vocab_size = vocab_size
    self.embeddings = nn.Embedding(
        vocab_size, d_model
    )

  def forward(self, x):
    return self.embeddings(x) * math.sqrt(self.d_model)

In [3]:
class PositionalEncoding(nn.Module):
  def __init__(self, d_model, max_seq_len):
    super().__init__()

    pe = torch.zeros(max_seq_len, d_model)
    position = torch.arange(0, max_seq_len, dtype=torch.float).unsqueeze(1)
    div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0)/d_model))

    pe[:, 0::2] = torch.sin(position * div_term)
    pe[:, 1::2] = torch.cos(position * div_term)

    self.register_buffer('pe', pe.unsqueeze(0))

  def forward(self, x):
    return x + self.pe[:, :x.size(1)]

In [4]:
class MultiHeadAttention(nn.Module):
  def __init__(self, d_model, num_heads):
    super().__init__()

    assert d_model % num_heads == 0, 'd_model must be divisible by num_heads.'

    self.num_heads = num_heads
    self.d_model = d_model
    self.head_dim = d_model // num_heads

    self.query_linear = nn.Linear(d_model, d_model, bias=False)
    self.key_linear = nn.Linear(d_model, d_model, bias=False)
    self.value_linear = nn.Linear(d_model, d_model, bias=False)

    self.output_linear = nn.Linear(d_model, d_model)

  def split_heads(self, x, batch_size):
    seq_len = x.size(1)
    x = x.reshape(batch_size, seq_len, self.num_heads, self.head_dim)

    return x.permute(0, 2, 1, 3)

  def compute_attention(self, query, key, value, mask=None):
    scores = torch.matmul(query, key.transpose(-2, -1)) / (self.head_dim ** 0.5)

    if mask is not None:
      scores = scores.masked_fill(mask == 0, float('-inf'))

    attention_weights = F.softmax(scores, dim=-1)

    return torch.matmul(attention_weights, value)

  def combine_heads(self, x, batch_size):
    x = x.permute(0, 2, 1, 3).contiguous()
    return x.view(batch_size, -1, self.d_model)

  def forward(self, q, k, v, mask=None):
    batch_size = q.size(0)

    query = self.split_heads(self.query_linear(q), batch_size)
    key = self.split_heads(self.key_linear(k), batch_size)
    value = self.split_heads(self.value_linear(v), batch_size)

    attention_weights = self.compute_attention(query, key, value, mask)

    output = self.combine_heads(attention_weights, batch_size)

    return self.output_linear(output)

In [5]:
class FeedForwardSubLayer(nn.Module):
  def __init__(self, d_model, d_ff):
    super().__init__()
    self.fc1 = nn.Linear(d_model, d_ff)
    self.fc2 = nn.Linear(d_ff, d_model)
    self.relu = nn.ReLU()

  def forward(self, x):
    return self.fc2(self.relu(self.fc1(x)))

In [ ]:
class EncoderLayer(nn.Module):
  def __init__(self, d_model, num_heads, d_ff, dropout):
    super().__init__()

    self.self_attn = MultiHeadAttention(d_model, num_heads)

    self.ff_sublayer = FeedForwardSubLayer(d_model, duff)

    self.norm1 = nn.LayerNorm(d_model)
    self.norm2 = nn.LayerNorm(d_model)

    self.dropout = nn.Dropout(dropout)

  def forward(self, x, src_mask):
    attn_output = self.self_attn(x, x, x, src_mask)

    x = self.norm1(x + self.dropout(attn_output))

    ff_output = self.ff_sublayer(x)

    x = self.norm2(x + self.dropout(ff_output))

    return x

class DecoderLayer(nn.Module):
  ...

In [ ]:
class TransformerEncoder(nn.Module):
  ...

class TransformerDecoder(nn.Module):
  ...

In [ ]:
class ClassificationHead(nn.Module):
  ...

In [ ]:
class Transformer(nn.Module):
  ...